## Part 3 Pythonic programming
>  话题提纲：decorator, iterator, generator, coroutine, asyncio, metaclass, concurrency

### 第九章　高级面向对象技术

#### 资产
Python提供另一种信息封装技术，替代访问器机制。该技术通过关键字property，将访问器包装成属性，使得访问器方法看上去和属性一样，再利用包装后的属性控制读取和修改。

In [3]:
%run examples/ooeg/prop.py

例子中size依然使用的是getSize, setSize读写属性。但从使用方式，size就像一个属性一样，而不是通过调用方法访问，访问细节被隐藏起来。
将property()函数返回的property对象，想象成一位代表，代言某个属性。对象的访问器被property包装后，用户想读取或修改属性值，就不必直接调用访问器，而要去联系它的代表，由代表负责读取和修改属性。

实际上，访问器方法依然需由程序员完成，property对象依然通过两个访问器读写属性。property的作用是将访问器的实现细节，对用户隐藏起来，户不必再去逐个记忆访问器方法名字，不必再去逐个记忆访问器方法需要的参数。记住属性的名字，就可以透明地对属性进行读取和修改。

##### 属性、方法、资产辨析
编程时，属性、方法和资产该使用哪个？属性和方法原本容易区分，不难选择。属性是数据、方法是操作数据的函数。资产技术模糊了属性和方法的界限。Python中数据属性、方法、资产，都可视为属性。可以这样理解，数据是数据属性，方法是可调用属性，而资产是定制属性。这或许可以帮助程序员决定，设计类和对象时，应该选用哪个。

具体说，方法代表对象发动的操作，方法做事情，方法名字一般是动词。如果属性不是动作，那就要决定，是用数据属性，还是资产。大多数情况下，都使用数据属性，数据属性一般是名词。若需要对数据属性进行访问控制，则需要将该数据属性转为资产。数据属性和资产的区别在于，当存取资产时，会调用特定的方法访问。下节我们看看，它有几个特别方法？

##### 细究资产
property()是一个内置函数，返回一个property对象。其详细信息如下。
- \pyth{property(fget=None, fset=None, fdel=None, doc=None)}
- fdel用于记录值被删除时情况
- docstring是说明，解释该property

通过一个例子看下这几个参数都是什么意思。

In [ ]:
class Silly:
    def _get_silly(self):
        print("Your are getting silly")
        return self._silly

    def _set_silly(self, value):
        print("You are making silly {}".format(value))
        self._silly = value

    def _del_silly(self):
        print("Whoah, you killed silly!")
        del self._silly

    silly = property(_get_silly, _set_silly, _del_silly, "This is a silly property")

s = Silly()
s.silly = "funny"
s.silly

help(s)
del s.silly

property()函数返回property对象。该对象通过“神奇方法（magic methods）”或“特别名称（special names）”的技术完成工作。所谓的特名，就是名字前后缀双下划线的方法，这样命名的方法一般是Python系统自用的，每个方法都有特定的功能，完成特定任务。

对property类而言，它的特名方法是\_\_get\_\_,\_\_set\_\_, \_\_delete\_\_()。这三个方法定义了描述器协议（descriptor protocol）。一个对象只要实现了这三个方法中任意一个，就可称为描述器（descriptor）。描述器特别处在于，它可以控制对属性的读写访问。而完成读写属性的，正是那三个特名方法。更详细技术说明，请阅读[descriptor](https://docs.python.org/3/howto/descriptor.html)。

我们对特名方法并不陌生，前面学过的\_\_init\_\_()、\_\_str\_\_()都是特名方法。同学们，能否说出这两个特名方法的功能是什么。

##### **装饰器**：另一种使用资产的方式
TODO：装饰器，装饰器实现继承、多态？

新的property语法，用装饰器（decorator）技术，返回property对象。装饰器通过语法糖衣（syntax sugar, constuctor like [])，将@property放在方法定义上一行，表明这个方法是某属性的描述器。这样做，免去给私有属性加下划线的麻烦，又可将多个方法整合为一个property。如此装饰访问器，语法上有一个要求，所有访问器方法必须同名。由装饰器名字，能判断出特名方法与访问器方法的对应关系，此外，property的docstring由getter方法中的注释指定。该类的property采用现代写法，行为没有变化，可读性更好了，代码看上去也更优雅。

TODO: 解释装饰器

In [ ]:
class Silly:
    @property
    def silly(self):  # getter
        "This is a silly property."  # docstring
        print("You are getting silly")
        return self._silly

    @silly.setter
    def silly(self, value): #  setter
        print("You are making silly {}".format(value))
        self._silly = value

    @silly.deleter
    def silly(self):  # deleter
        print("Whoah, you killed silly!")
        del self._silly
# Note all the methods share a common name.
s = Silly()
s.silly = "funny"
s.silly

help(s)
del s.silly

#### 类属性和类方法
装饰器技术在Python开发应用中有着广泛的应用，可以说随处可见。再举类方法和静态方法两个例子。

类方法（class methods）和静态方法(static methods)由装饰器@staticmethod和@classmethod，包装对象方法生成。类方法用cls指代该类，就像self指代某对象一样。

TODO:区别，何时用类方法，何时用静态方法？}

##### 阅读材料
- https://julien.danjou.info/blog/2013/guide-python-static-class-abstract-methods 
- https://stackoverflow.com/questions/136097/what-is-the-difference-between-staticmethod-and-classmethod-in-python
- https://stackoverflow.com/questions/12179271/meaning-of-classmethod-and-staticmethod-for-beginner
- https://www.geeksforgeeks.org/class-method-vs-static-method-python/

In [5]:
class MyClass:
    @staticmethod
    def smeth():
        print("This is a static method")

    @classmethod
    def cmeth(cls):
        print("This is a class method of", cls)

MyClass.smeth()
# This is a static method
MyClass.cmeth()
# This is a class method of <class '__main__.MyClass'>

This is a static method
This is a class method of <class '__main__.MyClass'>


#### 静态方法
Python中的静态方法主要是方便将外部函数集成到类体中， 重点在不需要类实例化的情况下调用方法。

In [10]:
# example waiting to debug
# IND = 'ON'

# class Kls(object):
#     """"""
#     def __init__(self, data):
#         """"""
#         self.data = data

#     @staticmethod
#     def checkind():
#         """"""
#         return (IND == 'ON')

#     def do_reset(self):
#         """"""
#         if checkind():
#             print('Reset done for: {}'.format(self.data))

#     def set_db(self):
#         """"""
#         if checkind():
#             self.db = 'New db connection'
#             print("DB connection made for: {}".format(self.data))

# ik1 = Kls(12)
# ik1.do_reset()
# ik1.set_db()
# print(ik1.checkind())
# print(Kls.checkind())

NameError: name 'checkind' is not defined

#### 多重继承
单继承指一个子类只有一个直接父类，多继承指一个子类有多个直接父类。Java支持单继承，C++、Python支持多继承。
##### 混入类
最简单和最常用的多继承形式，是所谓混入 (mixin) 类。混入类，指担任父类角色的类，一般不独立使用，而是希指望自己被其他类继承，由此，该父类的功能被扩展到其他类。

假设，我们设计一个通讯录类，里面有联系人姓名和电子邮。后来，我们继续设计，想让该通讯录类有发送电子邮件的功能。直接的想法是将此功能加入该通讯录类。更周详的思索会让我们应考虑到，不只通讯录类，其他类也可能需要发送电子邮件的功能，那么就应当设计一个具备发送电子邮件功能的混入类。任何类，只要想包含发电邮功能，继承此类即可。

再后来，我们希望给通讯录中的联系人增加居住地址。一般，地址是一个字符串，包含街、市、省、国等信息。我们有几个可供选择的设计方案。

- 将地址信息视为多个字符串参数，在实例化联系人对象时，传给构造器
- 将地址信息组织成元组或字典，传给构造器一个参数
- 创建地址类，地址信息是属性，把地址对象作为参数传给联系人对象构造器

三种方式，实际上，是选择将地址组织成何种数据类型供给联系人类使用。前两种选择是基本数据类型，第三种选择将地址抽象为自定义数据类型，类。其优势，一是可以增加方法，比如给出导航路线等。另一个优势，是其他需要地址的类，比如建筑物类、企业类、组织类可以通过继承重用地址类。通过组合两个类方式得到新类，技术上称为集聚（aggregation）或组合 (composition)。集聚标志两个类之间建立了从属（has a）关系。地址类从属于联系人类。

为了研究继承，我们使用继承来实现地址，创建一个AddressHolder类。
A Friend can have an Addres. Also, a Friend is an AddressHolder.

In [11]:
%run examples/ooeg/mixin.py

NameError: name 'Contact' is not defined

##### 菱形难题
当使用多重、多层继承时，多重继承会越来越麻烦。会遇到所谓菱形难题（diamond problem）。菱形难题怎么发生的呢？两个父类的都有\_\_init\_\_()方法。当这两个类都继承自object就出问题了。基类仅应被调用一次，那么，调用顺序是Friend->Contact->AddressHolder->Object，还是Friend->Contact->AddressHolder->Object？

##### 理解MRO
术上，调用顺序称为方法消解顺序(MRO, method resolution order)。根据MRO，可以找到解决菱形难题的方法。

首先看例子，菱形问题引起的错误。

In [ ]:
%run examples/ooeg/diamond2.py

##### 菱形难题解法
使用super可以解决菱形难题。
- Subclass的super().call_me()调用LeftSubclass.call_me()，进而调用其super().call_me()
- 但在这里super()没有指向LeftSubclass的基类BaseClass，而是指向RightSubclass, 尽管它不是LeftSubclass的父类。RightSubclass再调用BaseClass的call_me()
- 因此，多继承的super()找的并不是“父辈”方法，而是“后继(next)”方法
- 这个顺序是内置的mro机制决定的
- 结论：减少，乃至避免使用多重继承，除非有必要。调用父类方法时，不用直接调用语句，使用super()

In [ ]:
%run examples/ooeg/diamond3.py

#### 参数分组
参数分组，会让多继承问题进一步复杂。再看到联系人多继承例子。其\_\_init\_\_()方法里，参数被分组，分别传给了它的两个父类。


In [12]:
class Friend(Contact, AddressHolder):
    def __init__(self,
                name,
                email,
                phone,
                street,
                city,
                state,
                code):
        Contact.__init__(self, name, email)  # 1 group arguments
        AddressHolder.__init__(self, street, city, state,code)  # 2 groups of arguments
        self.phone = phone

NameError: name 'Contact' is not defined

按上节原则，调用父类方法时，不直接调用，而使用super()，那如何将参数分为不同组，传给super()呢？比如，super()先将name、 email传给Contact.\_\_init\_\_()，然后Contact.\_\_init\_\_()调用super，这就需要把地址信息传给“后继”方法，即AddressHolder.\_\_init\_\_()。当调用同名、不同参数的父类方法时，就会出现这个问题。最常出现这种问题处就是\_\_init\_\_()方法。

唯一解决办法，就是预先计划。设计基类参数列表时，基类用，而不一定每个子类都用的参数，定义为关键字参数。同时，定义该方法可以接受可变参数，并将这些参数传给它的super()方法。由此，根据MRO顺序，这些可变参数，可以适当地分组分配到父类的构造器里。
  
看下正规的多重继承代码示例。

In [13]:
%run examples/ooeg/multiinheri.py

#### Duck Typing
继承和多态的要旨是继承父类的共性方法，并能依据不同子类多态调用相应的方法。如果有这样一种技术，不需要继承，若干对象不必是某父类的子类，却可以共享一组方法。每个对象都提供这一组方法的实现，因此，不同的对象调用这组方法时，不会产生混淆。也就是说，不使用继承技术，却保有多态技术的优点。

有这样的方法！叫做Duck Typing，实在不好翻译，只好留着原文。

[Duck Typing](http://en.wikipedia.org/wiki/Duck_typing)是Python处理多态的方式，是一种不严格的多态。DT允许非子类的对象也可以调用多态方法。详细技术参考：[Duck Typing的解释](http://www.voidspace.org.uk/python/articles/duck_typing.shtlm)

> Don't check whether it is-a duck: check whether it quacks-like-a duck, walks-like-a duck, etc, etc, depending on exactly what subset of duck-like behavior you need to play you language-games with.(comp.lang.python, Jul. 26, 2000)  --Alex Martelli

> If it looks like a duck and quacks like a duck, it must be a duck.

> Duck typing in Python allows us to use /any/ object that provides the required behavior without forcing it to be a subclass。

> 用“鲸鱼”、“鳄鱼”解释duck typing，fishable-like whale。由于海棠花栽培历史悠久，深受人们喜爱，后来人们发现了与它花形相似的植物，往往就以海棠来命名，如秋海棠、四季海棠，丽格海棠等等。明代《群芳谱》所记载的“海棠四品”: 西府海棠，蔷薇科苹果属；垂丝海棠，蔷薇科苹果属；贴梗海棠，蔷薇科木瓜属；木瓜海棠，蔷薇科木瓜属。

> 因行为相似就归为一类，而不管其本来是不是一类

TODO: __len__(), __getitem__()}叫什么协议？？

面向对象程序设计技术广泛使用继承是因为有多态存在。在Python中，由于Duck Typing存在，降低了使用多态的必要程度，同时也降低了使用多继承的必要程度。

In [ ]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

beer_card = Card('7', 'diamonds')
beer_card

deck = FrenchDeck()
len(deck)

deck[0]
deck[-1]

from random import choice 
choice(deck)

deck[:3]
deck[12::13]

# for card in deck:  #  doctest: +ELLIPSIS
#     print(card)

# for card in reversed(deck):
#     print(card)

#### TODO: Iterator Pattern -- Dusty Phillips

##### Design patterns in brief

##### Comprehensions

##### Generators

##### Coroutines

##### Case study
##### Exercises

#### The decorator pattern

#### The observerpattern

#### Concurrency

##### Threads
##### Multiprocessing
##### AsyncIO

###### Case study
###### exercises

"Mastering python by Rick van Hattem"

#### functional programming - readability versus brevity

##### functional programming
##### lambda functions
- The Y combinator
##### itertools
- accumulate - 
-...
- islice

#### Decorators - enabling code reuse by decorating
- decorating functions
- decorating class functions
- decorating classes
- useful decorators

#### Generators and coroutines - infinity, one step at a time
##### What are generators?
##### Coroutines

#### Async IO - multithreading without threads
##### the asyncio library

##### Metaclasses - making classes(not instances) smarter

#### 对象表示法
用UML做面向对象分析与设计。自学
- [United Modeling Language](https://en.wikipedia.org/wiki/Unified_Modeling_Language)
- [统一建模语言](http://www.uml.org/what-is-uml.htm)
- [UML tutorial](https://www.w3cschool.cn/uml_tutorial)
- 《UML参考手册》

#### 练习: 简易记事本
UML设计图a simple command-line notebook application
\subsubsection{Analysis}
notes are short memos stored in a notebook.  Each note should record
the day it was written and can have tags added for easy querying.  It
should be possible to modify notes.  We also need to be able to search
for notes.  All these things should be done from the command line.
\subsubsection{Design}

Note object.  A notebook container object.  Tags and dates also seemto
be objects, but we can use dates from Python's standard library.

Note objects have attributes for memo, tags, creation\_date, and id.  A
match method for searching Notebook has the list of notes as an
attribute.  A search method that returns a list of filtered notes.
interact with these objects? Implement the command-line menu interface

#### 决策树，k-means

TODO:

%% \begin{framed}
%%   \begin{verbatim}
%% - 继承原则
%%   + Distinguish Interface Inheritance from Implementation Inheritance
%%     * /Inheritance of interface/ creates a subtype, implying an "is-a"
%%       relationship
%%     * /Inheritance of implementation/ avoids code duplication by reuse
%%   + Make Interfaces explicit with ABCs
%%   + Use Mixins for code Reuse
%%     *  Conceptually, a mixin does not define a new type; it merely
%%        bundles methods for reuse.  A mixin should never be
%%        instantiated, and concrete classes should not inherit only from
%%        a mixin.  Each mixin should provide a single specific behavior,
%%        implementing few and very closely related methods.
%%   + Make Mixins Explicit by Naming
%%   + An ABC may also be a mixin; the reverse is not true
%%   + Dont't subclass from more than one concrete class
%%   + Provide aggregate classes to users
%%     * If some combination of ABCs or mixins is particularly useful to
%%       client code, provide a class that brings them together in a
%%       sensible way
%%   + Favor object composition over class inheritance, don't overuse
%%     inheritance.  Because subclassing is a form of tight coupling, and
%%     tall inheritance trees tend to be brittle. 
%% \end{verbatim}
%% \end{framed}

    
% **** 通过继承扩展内置数据类型

% - python所有类都继承自一个特殊类object
% - 如果自定义类继承自其他数据类型，会怎样？比如上例，如果想在通讯列表中
%   搜索姓名，要怎么办？
% - 可以在Contact类中增加Search方法，但Search似乎应该是附于list的方法，而
%   不是附于Contact
% - 用继承可以解决这个问题
% - pythonic方式是使用duck typing，定义Contact为iterable，本例为讲继承而
%   使用继承机制

% #+begin_src python :results output
% class ContactList(list):
%     """
%     Contactlist是list子类，因此继承list的方法，并增加了list没有，而自己
%     独有的方法search()
%     """
%     def search(self, name):
%         """Return all contacts that contain the search value in their name."""
%         matching_contacts = []
%         #  the syntax [] is called syntax sugar that calls the list() constructor.
%         # [] == list()
%         # isinstance([], object)
%         for contact in self:
%             if name in contact.name:
%                 matching_contacts.append(contact)
%         return matching_contacts


% class Contact:
%     all_contacts = ContactList()  #  类变量，是一个扩展的list

%     def __init__(self, name, email):
%         self.name = name
%         self.email = email
%         Contact.all_contacts.append(self)


% class Supplier(Contact):
%     def order(self, order):
%         print("If this were a real system we would send '{}' order to '{}'".format(order, self.name))


% c1 = Contact("John A", "johna@ex.net")
% c2 = Contact("John B", "johnb@ex.net")
% c3 = Contact("Jenna C", "jennac@ex.net")

% [c.name for c in Contact.all_contacts.search("John")]
% #+end_src

% - 可以扩展python的各种内置数据类型，为自己项目所用，比如：字典
% - 标准的python用法，不直接继承built-in type
% - collections模块提供了Userdict, UserList, UserString。想继承内置类型性质，继承这几个
% - fluent python中讲了继承内置类型可能引发的问题

% #+begin_src python :results output
% # 仅示例，不推荐这种用法
% class LongNameDict(dict):
%     """继承字典，既包括字典属性和方法，又增加独有方法longest_key()"""
%     def longest_key(self):
%         longest = None
%         for key in self:
%             if (not longest) or (len(key) > len(longest)):
%                 longest = key
%         return longest

% longkeys = LongNameDict()
% longkeys['hello'] = 1
% longkeys['longest yet'] = 5
% longkeys['hello2'] = 'world'

% longkeys.longest_key()
% #+end_sr

\section{虚基类?}

%% - 与dock typing常伴生的一项技术是虚基类 (ABCs, Abstract Base Classes)。
%%   不展开，留自学
%% - 大部分虚基类都放在python标准库的collections.abc模块中
%% - https://docs.python.org/3/library/collections.abc.html?highlight=collections%20abc#module-collections.abc
%% - 例:

%% #+begin_src python :results output
%% from collections import Container
%% Container.__abstractmethods__
%% #  frozenset({'__contains__'})
%% help(Container.__contains__) 

%% # 使用虚基类
%% class OddContainer:
%%     def __contains__(self, x):
%%         if (not isinstance(x, int)) or (not x%2):
%%             return False
%%         return True

%% oc = OddContainer()
%% #  类型判断
%% #  某个自定义类，duck typing了某个虚基类的protocol，就可以断定它是该虚基类的实例或子类。
%% #  勿须继承自该虚基类
%% isinstance(oc, Container)
%% issubclass(OddContainer, Container)
%% #+end_src
%% 不通过继承（或多重继承），dt创建了一种is a关系。

\section{练习: 简易记事本}
UML设计图a simple command-line notebook application
\subsubsection{Analysis}
notes are short memos stored in a notebook.  Each note should record
the day it was written and can have tags added for easy querying.  It
should be possible to modify notes.  We also need to be able to search
for notes.  All these things should be done from the command line.
\subsubsection{Design}

Note object.  A notebook container object.  Tags and dates also seemto
be objects, but we can use dates from Python's standard library.

Note objects have attributes for memo, tags, creation\_date, and id.  A
match method for searching Notebook has the list of notes as an
attribute.  A search method that returns a list of filtered notes.
interact with these objects? Implement the command-line menu interface



% *** 案例2： 房地产管理应用程序

% - 创建一个房地产管理应用，让中介管理出售和出租的资产
%   + 有两种资产：公寓和独栋。中介录入资产信息，列出所有可用资产，标记资
%     产为已售或已租。资产一旦售出就不考虑编辑更新其信息了
%   + 使用python命令行界面管理应用
% - 细读需求，可以找出些许名词，它们或许就是潜在的应用里的类。需要表示资
%   产，独栋和公寓是两个类，租和售也不同
% - 主要考虑使用继承技术
% - House, Apartment是Property的子类，Property是父类
% - UML设计图
% [[file:./py/agent.py][源文件]]

% *** 案例3: 计算多边形边长

% - 面向对象分析设计，何时使用oop?
% - 面向对象编程范式下，分析、设计人员
%   + 首先识别出问题领域(domain-driven programming)的对象
%   + 再识别、设计每个对象的属性、方法，及对象间交互联系
%   + 通过以上步骤，为问题领域建模
% - 对象既有数据（属性），又有行为（方法）
%   + 如果仅需要数据，完全可以将数据存储到列表、集合、字典中
%   + 如果仅需要行为，而不关心数据存储，那函数就够了
% - 一般编程思路
%   + 首先会想到使用哪些数据，会把这些数据存入变量
%   + 慢慢地，会发现，总会把相关的一组数据传入某些函数处理
%   + 这时，就应该考虑，把这些变量、函数组合起来，进一步抽象为一个对象
%   + 慢慢的，会发现对象之间的关系，考虑将对象归类，将类构造为继承层次
% - 此时，应该回到分析之出，假设自己，一开始就考虑问题领域里应该有哪些对
%   象。逐渐的，习惯这种思路之后，遇到问题后，首先会考虑，问题领域中有哪
%   些对象，这些对象如何交互实现了各种功能。这就是面向对象分析与设计
% - 举例： 计算多边形边长，两种编程范式对比
%   + 注意代码可读性。
% - 源码: [[file:./py/polygons.py][计算多边形边长]]

% *** 案例4: Modeling a /Document/ class that might be used in a text editor or word processor

% - What objects, funcitons, or properties should it have?
% - ./py/docu
